## Tractography

Tractography is a local fiber tracking technique employed to model axonal trajectories as streamlines from local directional information. There are a number of algorithms to perform tractography. Broadly, these algorithms can be categorized into two types: (1) deterministic and (2) probabilistic. In this notebook, we will follow cover the basics of deterministic tractography. Deterministic algorithms perform tracking of streamlines by following a predictable path, such as following the primary diffusion direction ($\lambda_1$). Most tractography algorihms follows 2 general principles - (1) estimate the fiber orientation, and (2) follow along these orientations to generate the streamline.

Before we begin, we should note that this lesson requires a

To perform fiber tracking, three things are needed:
    1. A method for getting directions from a diffusion data set (eg. diffusion tensor)
    2. Method for identifying when to stop tracking
    3. Set of seeds from which to begin tracking.

First, we will quickly repeat the steps from the previous notebook and compute the diffusion tensor. We will additionally extract the `affine` data from the diffusion image, which we will need later on! 

In [1]:
from bids.layout import BIDSLayout
from dipy.io.gradients import read_bvals_bvecs
from dipy.core.gradients import gradient_table
from nilearn import image as img
import nibabel as nib

layout = BIDSLayout("../data/ds000030/derivatives", validate=False)

dwi = layout.get(subject='10788', suffix='preproc', extension='nii.gz', return_type='file')[0]
bval = layout.get(subject='10788', suffix='preproc', extension='bval', return_type='file')[0]
bvec = layout.get(subject='10788', suffix='preproc', extension='bvec', return_type='file')[0]

dwi_img = img.load_img(dwi)
affine = dwi_img.affine

gt_bvals, gt_bvecs = read_bvals_bvecs(bval, bvec)
gtab = gradient_table(gt_bvals, gt_bvecs)

In [2]:
import dipy.reconst.dti as dti
from dipy.segment.mask import median_otsu

dwi_data = dwi_img.get_data()
dwi_data, dwi_mask = median_otsu(dwi_data, vol_idx=[0], numpass=1) # Specify the volume index to the b0 volumes

dti_model = dti.TensorModel(gtab)
dti_fit = dti_model.fit(dwi_data, mask=dwi_mask) # This step may take a while

We will perform tracking using a deterministic algorithm on tensor fields via `EuDX` [(Garyfallidis _et al._, 2012)](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3518823/). `EuDX` makes use of the primary direction of the diffusion tensor to propogate streamlines from voxel to voxel and a stopping criteria from the fractional anisotropy (FA). We will get the FA map and eigenvectors from our tensor fitting.

In [3]:
fa_img = dti_fit.fa
evecs_img = dti_fit.evecs

In the background of the image, the fitting may not be accurate as all of the measured signal is primarily noise and it is possible that values of nans (not a number) may be found in the FA map. We can remove these using `numpy` to find and set these voxels to 0

In [4]:
import numpy as np

fa_img[??] = ??

One of the inputs of `EuDX` is the discretized voxel directions on a unit sphere. Therefore, it is necessary to discretize the eigenvectors before providing them to `EuDX`. We will use an evenly distributed sphere of 362 points using the `get_sphere` function.

In [6]:
from dipy.data import ??

sphere = ??(??)

We will determine the indices representing the discretized directions of the peaks by providing as input, our tensor model, the diffusion data, the sphere, and a mask to apply the processing to. Additionally, we will set the minimum angle between directions, the maximum number of peaks to return (1 for the tensor model), and the relative peak threshold (returning peaks greater than this value).

_Note: This step may take a while to run_

In [7]:
from dipy.direction import ??

peak_indices = ??(model=??, data=??, sphere=??, relative_peak_threshold=??, min_separation_angle=??, mask=??, npeaks=??)

Additionally, we will apply a stopping criterion for our tracking based on the FA map. That is, we will stop our tracking when we reach a voxel where FA is below 0.2.

In [8]:
from dipy.tracking.stopping_criterion import ??

stopping_criterion = ??(??, ??)

We will also need to specify where to "seed" (begin) the fiber tracking. Generally, the seeds chosen will depend on the pathways one is interested in modelling. In this example, we will create a seed mask from the FA map thresholding above our stopping criterion.

In [9]:
from dipy.tracking import ??

seed_mask = fa_img.copy()
seed_mask[seed_mask>=??] = ??
seed_mask[seed_mask<??] = ??

seeds = ??.seeds_from_mask(??, affine=??, density=??)

Now, we can apply the tracking algorithm! 

As mentioned previously, `EuDX` is the fiber tracking algorithm that we will be using. The most important parameters to include are the indices representing the discretized directions of the peaks (`peak_indices`), the stopping criterion, the seeds, the affine transformation, and the step sizes to take when tracking!

In [10]:
from dipy.tracking.local_tracking import ??
from dipy.tracking.streamline import ??

# Initialize local tracking - computation happens in the next step.
streamlines_generator = ??(??, ??, ??, affine=??, step_size=??)

# Generate streamlines object
streamlines = ??(??)

We just created a deterministic set of streamlines using the `EuDX` algorithm mapping the human connectome (tractography). We can save the streamlines as a Trackvis file so it can be loaded into other software for visualization or further analysis. To do so, we need to save the tractogram state using `StatefulTractogram` and `save_trk` to save the file. Note that we will have to specify the space to save the tractogram in.

In [11]:
from dipy.io.stateful_tractogram import ??
from dipy.io.streamline import ??

sft = StatefulTractogram(??, ??, ??)
save_trk(??, ??)

We can then display the resulting streamlines using the `fury` python package

In [1]:
from dipy.io.streamline import load_trk

sft = load_trk(??, ??)

In [3]:
sft.to_vox()
streamlines = sft.streamlines

In [4]:
from dipy.viz import window, actor, colormap

# Prepare display objects
# streamlines_actor = actor.line(streamlines, colormap.line_colors(streamlines))
streamlines_actor = actor.line(streamlines)

# Create 3D display
r = window.Renderer()
r.add(streamlines_actor)

# Save still images
window.record(r, out_path=??, size=(800,800))

/home/tkai/.venv/dti_venv/lib/python3.6/site-packages/fury/colormap.py:229: RuntimeWarning: invalid value encountered in true_divide
  orient = np.abs(orient / np.linalg.norm(orient))
